# Math of VoteKit

This notebook is an aggregation of the tutorial notebooks from our documentation page. This is intended to focus on the underlying math of VoteKit, and should be used as a companion to the "Math of VoteKit" presentation. This notebook is quite long, just a warning.


## Ballots
The first order of business is ballots. In the context of ranked choice voting, a ballot records a voter's preferences as a linear ordering of the candidates. If a voter casts the ballot $A>B>C$, this means they prefer candidate $A$ to $B$ and candidate $B$ to $C$. (We often use the $>$ symbol to indicate higher preference.) Let's see how ballots are stored in VoteKit. First we import the necessary modules.

In [1]:
from votekit.ballot import Ballot
from fractions import Fraction

ballot = Ballot(ranking=[{"A"}, {"B"}, {"C"}], weight=Fraction(3, 2))
print(ballot)

Ranking
1.) A, 
2.) B, 
3.) C, 
Weight: 3/2


Here, we have created one ballot. The ballot stored the ranking $A>B>C$. The weight attribute indicates "how many" of this ballot were cast. It defaults to 1, but we have put in 3/2.  A fractional weight will be useful in single transferable vote (STV) elections! While the ballot stores the weight as a `Fraction` type, you can actually input the weight as an integer or float and it will convert it for you.



In [2]:
ballot = Ballot(ranking=[{"A"}, {"B"}, {"C"}], weight=3 / 2)
print(ballot)

ballot = Ballot(ranking=[{"A"}, {"B"}, {"C"}], weight=32)
print(ballot)

Ranking
1.) A, 
2.) B, 
3.) C, 
Weight: 3/2
Ranking
1.) A, 
2.) B, 
3.) C, 
Weight: 32


#### **Try it yourself**
Create three new ballots, one with weight 47, one with weight 22/7, and one with your own weight idea.


In [3]:
# your code here

Full linear rankings are not the only possible ballots.
Real-world voters frequently list multiple candidates in the same position (even if that is against the rules of the election). As far as we know, this is always treated by localities running ranked choice elections as a voter error, called an **overvote**.

Voters also leave some candidates out. In an extreme case, when a voter only lists one candidate, we call this a **bullet vote**.  These are fairly common in ranked elections, and a position with no candidates listed is sometimes called an **undervote**.

We might prefer for localities running ranked choice elections to be smart about the voter intent to communicate a tied preference -- and we can do that in VoteKit.  But we'll get to running elections later.

In [ ]:
ballot = Ballot(ranking=[{"A", "D"}, {"B", "B", "B"}, {"C", "E", "F", "B"}])
print("A ballot with overvotes:", ballot)

A ballot with overvotes: Ranking
1.) D, A, (tie)
2.) B, 
3.) B, C, E, F, (tie)
Weight: 1


The ballot above says that candidates $D$ and $A$ were ranked first, $B$ second, and $E,C,F$ all in third.

In [ ]:
ballot = Ballot(ranking=[{"B"}])
print("A bullet vote:")
print(ballot)

A bullet vote:
Ranking
1.) B, 
Weight: 1


The ballot above is a bullet vote; only candidate $B$ is listed in first.

#### **Automatic cleaning vs specified cleaning**
What we really mean to illustrate above is that the `Ballot` class has no understanding of the rules of your election. It is flexible enough to allow all sorts of rankings, even ones that are not valid. 

Since the ranking is a list of *sets*, the only default cleaning that occurs in ballots in VoteKit is that the candidates listed in a particular position will be deduplicated. In the code above, the first ballot should only print with one candidate named "B" in position two. 

There are many other kinds of cleaning functions, but you have to choose to apply those yourself. This is really crucial to know; lots of elections will behave strangely if you do not have the correct ballot types as input, but it is up to you to clean them to the level needed for your method of election.

## Preference Profiles

When we want to aggregate a collection of ballots cast by voters, we use the `PreferenceProfile` object. It stores all of the ballots, allows us to visualize them, and comes with some handy features. 

First we display the simple profile, which just repeats the weights as they were inputted.

In [7]:
from votekit.pref_profile import PreferenceProfile

candidates = ["A", "B", "C"]

# let's assume that the ballots come from voters,
# so they all have integer weight for now
ballots = [
    Ballot(ranking=[{"A"}, {"B"}, {"C"}], weight=3),
    Ballot(ranking=[{"B"}, {"A"}, {"C"}]),
    Ballot(ranking=[{"C"}, {"B"}, {"A"}]),
    Ballot(ranking=[{"A"}, {"B"}, {"C"}]),
    Ballot(ranking=[{"A"}, {"B"}, {"C"}]),
    Ballot(ranking=[{"B"}, {"A"}, {"C"}]),
]

# we give the profile a list of ballots and a list of candidates
profile = PreferenceProfile(ballots=ballots, candidates=candidates)

print(profile.df.to_string())

             Ranking_1 Ranking_2 Ranking_3 Voter Set Weight
Ballot Index                                               
0                  (A)       (B)       (C)        {}      3
1                  (B)       (A)       (C)        {}      1
2                  (C)       (B)       (A)        {}      1
3                  (A)       (B)       (C)        {}      1
4                  (A)       (B)       (C)        {}      1
5                  (B)       (A)       (C)        {}      1


The `PreferenceProfile` class takes a list of `Ballot` objects and a list of candidates. The candidate names must be distinct, and it will raise an error if not. Providing the list of candidates is actually optional, and it has no impact on the profile object. If the candidates are not provided, the profile automatically computes the candidates as anyone who appeared on a ballot with positive weight.  However, later when we move on to ballot generation, the list of candidates will be important, so it is good practice to specify them.


Notice that printing the profile did not automatically combine like ballots into a single line. But there's an easy way to get the grouped profile, as follows.

In [8]:
grouped_profile = profile.group_ballots()
print(grouped_profile.df.to_string())

             Ranking_1 Ranking_2 Ranking_3 Weight Voter Set
Ballot Index                                               
0                  (A)       (B)       (C)      5        {}
1                  (B)       (A)       (C)      2        {}
2                  (C)       (B)       (A)      1        {}


A few other useful attributes/methods are listed here.  Use `profile.ATTR` for each one.

- `candidates` returns the list of candidates input to the profile.

- `candidates_cast` returns the list of candidates who received votes.

- `ballots` returns the list of ballots (useful if you want to extract the ballots to write custom code, say).

- `num_ballots` returns the number of ballots, which is the length of `ballots`.

- `total_ballot_wt` returns the sum of the ballot weights.




### **Try it yourself**
Try using all of the above attributes/methods, with or without grouping the ballots.




## Preference Intervals

There are a few ways to input ballots into VoteKit. You can type them all by hand as we did above, you can read them in from real-world vote records, or you can generate them within VoteKit. While we will dive a lot deeper into reading and generating in future sections, it is worthwhile to introduce some of the vocabulary surrounding generative models here.

Most of our generative models rely on a **preference interval**. A preference interval stores information about the relative strengths of a voter's priorities for candidates. We visualize this, unsurprisingly, as an interval. We take the interval $[0,1]$ and divide it into pieces, where we've taken all the preference weights and scaled so they add to 1.

For example,
```
{"A": 0.7, "B": 0.2, "C": 0.1}
```
is a dictionary that represents an ordered preference interval where A is preferred to B by a ratio of 7:2, etc.  

Later, the ballot generator models will pull from these preferences to create a ballot for each voter.

It should be remarked that there is a difference, at least to VoteKit, between the intervals
```
{"A": 0.7, "B": 0.3, "C": 0} and
{"A": 0.7, "B": 0.3}
```

While both say there is no preference for candidate C, if the latter interval is fed into VoteKit, that third candidate will never appear on a generated ballot. If we feed it the former interval, the third candidate will appear at the bottom of the ballot.

![png](https://votekit.readthedocs.io/en/latest/_images/preference_interval.png)

One of the generative models is called the **slate-Plackett-Luce model**, or s-PL.  In s-PL, voters fill in their ballot from the top position to the bottom by choosing from the available candidates in proportion to their preference weights. We call this the impulsive voter model.

You can read more about s-PL in our [social choice documentation](https://votekit.readthedocs.io/en/latest/social_choice_docs/scr/#slate-plackett-luce), but for now let's use it to explore how intervals work. We will assume there is only one bloc of voters. This makes the syntax look a little strange, but bear with us.

In [10]:
import votekit.ballot_generator as bg
from votekit import PreferenceInterval

# the sPL model assumes there are blocs of voters,
# but we can just say that there is only one bloc
bloc_voter_prop = {"all_voters": 1}
slate_to_candidates = {"all_voters": ["A", "B", "C"]}

# the preference interval (80,15,5)
pref_intervals_by_bloc = {
    "all_voters": {"all_voters": PreferenceInterval({"A": 0.80, "B": 0.15, "C": 0.05})}
}

# the sPL model needs an estimate of cohesion between blocs,
# but there is only one bloc here
cohesion_parameters = {"all_voters": {"all_voters": 1}}

pl = bg.slate_PlackettLuce(
    pref_intervals_by_bloc=pref_intervals_by_bloc,
    bloc_voter_prop=bloc_voter_prop,
    slate_to_candidates=slate_to_candidates,
    cohesion_parameters=cohesion_parameters,
)

profile = pl.generate_profile(number_of_ballots=100)
print(profile)
print()
print(profile.df.to_string())

Profile contains rankings: True
Maximum ranking length: 3
Profile contains scores: False
Candidates: ('A', 'C', 'B')
Candidates who received votes: ('A', 'B', 'C')
Total number of Ballot objects: 6
Total weight of Ballot objects: 100


             Ranking_1 Ranking_2 Ranking_3 Voter Set Weight
Ballot Index                                               
0                  (A)       (B)       (C)        {}     66
1                  (A)       (C)       (B)        {}     15
2                  (B)       (A)       (C)        {}     14
3                  (B)       (C)       (A)        {}      1
4                  (C)       (B)       (A)        {}      1
5                  (C)       (A)       (B)        {}      3


Re-run the above block several times to see that the elections will come out different!  The s-PL model is random, meaning we won't always get the same profile when we run `generate_profile` (although we are planning to implement an explicit `random seed` option so that you can replicate runs). You probably won't get the same output as what is stored in this tutorial either. That's okay! Check that most ballots rank $A$ first, which is expected because they had the largest portion of the preference interval. Likewise, $C$ is least popular.

## Blocs

A **bloc** of voters is a group of voters who have similar voting behavior, generally preferring their **slate** of candidates to the slates associated to other blocs. In VoteKit, we model this by assuming voters within a bloc have the same preference interval. Let's look at an example where there are two blocs called Alpha and Xenon, each with a two-candidate slate ($A,B$ and $X,Y$, respectively). 

By introducing blocs, we also need to discuss cohesion parameters. In realistic polarized elections, we might be able to identify two groups with different voting tendencies, but real voting blocs are not perfectly monolithic---some voters will stick with their slate, but many others might have a tendency to "cross over" to the other slate sometimes in constructing their ballot.

The precise meaning of these vary by model, but broadly speaking, **cohesion parameters** measure the strength with which voters within a particular bloc stick to their slate.

Since we are printing out a large profile below, we will also introduce the `profile_df_head` method which prints the top ballots by weight.

In [12]:
from votekit.pref_profile import profile_df_head
slate_to_candidates = {"Alpha": ["A", "B"], "Xenon": ["X", "Y"]}

# note that we include candidates with 0 support,
# and that our preference intervals will automatically rescale to sum to 1

pref_intervals_by_bloc = {
    "Alpha": {
        "Alpha": PreferenceInterval({"A": 0.8, "B": 0.2}),
        "Xenon": PreferenceInterval({"X": 0, "Y": 1}),
    },
    "Xenon": {
        "Alpha": PreferenceInterval({"A": 0.5, "B": 0.5}),
        "Xenon": PreferenceInterval({"X": 0.5, "Y": 0.5}),
    },
}


bloc_voter_prop = {"Alpha": 0.8, "Xenon": 0.2}

# assume that each bloc is 90% cohesive
# we'll discuss exactly what that means later
cohesion_parameters = {
    "Alpha": {"Alpha": 0.9, "Xenon": 0.1},
    "Xenon": {"Xenon": 0.9, "Alpha": 0.1},
}

pl = bg.slate_PlackettLuce(
    pref_intervals_by_bloc=pref_intervals_by_bloc,
    bloc_voter_prop=bloc_voter_prop,
    slate_to_candidates=slate_to_candidates,
    cohesion_parameters=cohesion_parameters,
)

# the by_bloc parameter allows us to see which ballots came from which blocs of voters
profile_dict, agg_profile = pl.generate_profile(number_of_ballots=10000, by_bloc=True)
print("The ballots from Alpha voters\n", profile_df_head(profile_dict["Alpha"], 10))

print("The ballots from Xenon voters\n", profile_df_head(profile_dict["Xenon"],10))

print("Aggregated ballots\n", profile_df_head(agg_profile,10))

The ballots from Alpha voters
              Ranking_1 Ranking_2 Ranking_3 Ranking_4 Weight Voter Set
Ballot Index                                                         
0                  (A)       (B)       (Y)       (X)   5188        {}
4                  (B)       (A)       (Y)       (X)   1289        {}
3                  (Y)       (A)       (B)       (X)    665        {}
1                  (A)       (Y)       (B)       (X)    546        {}
2                  (Y)       (B)       (A)       (X)    177        {}
5                  (B)       (Y)       (A)       (X)    135        {}
The ballots from Xenon voters
              Ranking_1 Ranking_2 Ranking_3 Ranking_4 Weight Voter Set
Ballot Index                                                         
1                  (X)       (Y)       (B)       (A)    420        {}
7                  (Y)       (X)       (B)       (A)    407        {}
6                  (Y)       (X)       (A)       (B)    404        {}
0                  (X)      

Scan this to be sure it is reasonable, recalling that our intervals say that the Alpha voters prefer $A$ to $B$, while $X$ has no support in that bloc. Xenon voters like $X$ and $Y$ equally, and then like $A$ and $B$ equally (although much less than their own slate). There should be a lot more Alpha-style voters than Xenon-style voters.



## Elections

Finally, we are ready to run an election. It is important to distinguish between *preference profiles*, which are a collection of ballots, and *elections*, which are the method by which those ballots are converted to an outcome (candidates elected to seats). We will explore all sorts of election types in later notebooks. For now, let's use a plurality election on a small set of ballots so we can verify that it behaves as it should.

In [14]:
from votekit.elections import Plurality

ballots = [
    Ballot(ranking=[{"A"}, {"B"}, {"C"}]),
    Ballot(ranking=[{"B"}, {"A"}, {"C"}]),
    Ballot(ranking=[{"C"}, {"B"}, {"A"}]),
    Ballot(ranking=[{"A"}, {"B"}, {"C"}]),
    Ballot(ranking=[{"A"}, {"B"}, {"C"}]),
    Ballot(ranking=[{"B"}, {"A"}, {"C"}]),
]

profile = PreferenceProfile(ballots=ballots * 6, candidates=candidates)

profile = profile.group_ballots()

print(profile_df_head(profile,10))

# m is the number of seats to elect
election = Plurality(profile=profile, m=1)

print(election)

             Ranking_1 Ranking_2 Ranking_3 Weight Voter Set
Ballot Index                                               
0                  (A)       (B)       (C)     18        {}
1                  (B)       (A)       (C)     12        {}
2                  (C)       (B)       (A)      6        {}
      Status  Round
A    Elected      1
B  Remaining      1
C  Remaining      1


If everything worked as intended, you should see that $A$ was elected, while $B,C$ were remaining. There is only one round, as plurality elections are single step.



You can also run a plurality election with more seats than one; it just takes the $m$ candidates with the most first-place support as winners.

For advanced users:  if several candidates had the same level of first-place support, the default tiebreaker in VoteKit is `None`, and it will raise an error telling you to choose a tiebreak method. This can be done by setting `tiebreak='random'` or `tiebreak='borda'` in the `Plurality` init method. There is also a `'first_place'` option, but that won't help in a plurality tie.

### Conclusion
The goal of this section was to introduce the vocabulary of VoteKit and ranked choice voting. You should now know about ballots, preference profiles, preference intervals, blocs/slates, and the distinction between profiles and elections. 

#### Extra Prompts

If you have finished this section and are looking to extend your understanding, try the following prompts:

- Write your own profile with four candidates named Trump, Rubio, Cruz, and Kasich, a preference interval of your choice, and with the bloc name set to "Repubs2016". Generate 1000 ballots. Are they distributed how they should be given your preference interval?
- Create a preference profile where candidates $B,C$ should be elected under a 2-seat plurality election. Run the election and confirm!

## Simulated voting with ballot generators

If we want to get a large sample of ballots without using real-world data, we can use a variety of ballot generators included in VoteKit.

### Bradley-Terry

The slate-Bradley-Terry model (s-BT) uses the same set of input parameters as s-PL: `slate_to_candidates`, `bloc_voter_prop`, `cohesion_parameters`, and `pref_intervals_by_bloc`. We call s-BT the deliberative voter model because part of the generation process involves making all pairwise comparisons between candidates on the ballot. A more detailed discussion can be found in our [social choice documentation](../../social_choice_docs/scr.html#slate-bradley-terry).




In [16]:
import votekit.ballot_generator as bg
from votekit import PreferenceInterval

slate_to_candidates = {"Alpha": ["A", "B"], "Xenon": ["X", "Y"]}

# note that we include candidates with 0 support, and that our preference intervals
# will automatically rescale to sum to 1

pref_intervals_by_bloc = {
    "Alpha": {
        "Alpha": PreferenceInterval({"A": 0.8, "B": 0.15}),
        "Xenon": PreferenceInterval({"X": 0, "Y": 0.05}),
    },
    "Xenon": {
        "Alpha": PreferenceInterval({"A": 0.05, "B": 0.05}),
        "Xenon": PreferenceInterval({"X": 0.45, "Y": 0.45}),
    },
}


bloc_voter_prop = {"Alpha": 0.8, "Xenon": 0.2}

# assume that each bloc is 90% cohesive
cohesion_parameters = {
    "Alpha": {"Alpha": 0.9, "Xenon": 0.1},
    "Xenon": {"Xenon": 0.9, "Alpha": 0.1},
}

bt = bg.slate_BradleyTerry(
    pref_intervals_by_bloc=pref_intervals_by_bloc,
    bloc_voter_prop=bloc_voter_prop,
    slate_to_candidates=slate_to_candidates,
    cohesion_parameters=cohesion_parameters,
)

profile = bt.generate_profile(number_of_ballots=100)
print(profile_df_head(profile,10))

             Ranking_1 Ranking_2 Ranking_3 Ranking_4 Voter Set Weight
Ballot Index                                                         
0                  (A)       (B)       (Y)       (X)        {}     58
3                  (B)       (A)       (Y)       (X)        {}     10
5                  (X)       (Y)       (A)       (B)        {}      8
1                  (A)       (Y)       (B)       (X)        {}      7
8                  (Y)       (X)       (B)       (A)        {}      5
4                  (Y)       (A)       (B)       (X)        {}      3
6                  (X)       (Y)       (B)       (A)        {}      3
7                  (Y)       (X)       (A)       (B)        {}      3
2                  (B)       (Y)       (A)       (X)        {}      2
9                  (A)       (X)       (B)       (Y)        {}      1


## Generating Preference Intervals from Hyperparameters

Now that we have seen a few ballot generators, we can introduce the candidate simplex and the Dirichlet distribution.


We saw that you can initialize the Plackett-Luce model and the Bradley-Terry model from a preference interval (or multiple ones if you have different voting blocs). Recall, a preference interval stores a voter's preference for candidates as a vector of non-negative values that sum to 1. Other models that rely on preference intervals include the Alternating Crossover model (AC) and the Cambridge Sampler (CS). There is a nice geometric representation of preference intervals via the candidate simplex.

### Candidate Simplex

Informally, the candidate simplex is a geometric representation of the space of preference intervals. With two candidates, it is an interval; with three candidates, it is a triangle; with four, a tetrahedron; and so on getting harder to visualize as the dimension goes up.

This will be easiest to visualize with three candidates $A,B,C$. Then there is a one-to-one correspondence between positions in the triangle and what are called **convex combinations** of the extreme points.  For instance, $.8A+.15B+.05C$ is a weighted average of those points giving 80% of the weight to $A$, 15% to $B$, and 5% to $C$.  The result is a point that is closest to $A$, as seen in the picture.  

Those coefficients, which sum to 1, become the lengths of the candidate's sub-intervals.  So this lets us see the simplex as the space of all preference intervals.

![png](https://votekit.readthedocs.io/en/latest/_images/candidate_simplex.png)

### Dirichlet Distribution

**Dirichlet distributions** are a one-parameter family of probability distributions on the simplex---this is used here to choose a preference interval at random. We parameterize it with a value $\alpha \in (0,\infty)$. As $\alpha\to \infty$, the support of the distribution moves to the center of the simplex. This means we are more likely to sample preference intervals that have roughly equal support for all candidates, which will translate to all orderings being equally likely. As $\alpha\to 0$, the mass moves to the vertices. This means we are more likely to choose a preference interval that has strong support for a single candidate.  In between is $\alpha=1$, where any region of the simplex is weighted in proportion to its area.  We think of this as the "all bets are off" setting -- you might choose a balanced preference, a concentrated preference, or something in between.

The value $\alpha$ is never allowed to equal 0 or $\infty$ in Python, so VoteKit changes these to a very small number ($10^{-10}$) and a very large number $(10^{20})$.  We don't recommend using values that extreme. In previous studies, MGGG members have taken $\alpha = 1/2$ to be "small" and $\alpha = 2$ to be "big."

![png](https://votekit.readthedocs.io/en/latest/_images/dirichlet_distribution.png)

It is easy to sample a `PreferenceInterval` from the Dirichlet distribution. Rerun the code below several times to get a feel for how these change with randomness.

In [ ]:
strong_pref_interval = PreferenceInterval.from_dirichlet(
    candidates=["A", "B", "C"], alpha=0.1
)
print("Strong preference for one candidate", strong_pref_interval)

abo_pref_interval = PreferenceInterval.from_dirichlet(
    candidates=["A", "B", "C"], alpha=1
)
print("All bets are off preference", abo_pref_interval)

unif_pref_interval = PreferenceInterval.from_dirichlet(
    candidates=["A", "B", "C"], alpha=10
)
print("Uniform preference for all candidates", unif_pref_interval)

Strong preference for one candidate {'A': 0.1521, 'B': 0.0002, 'C': 0.8477}
All bets are off preference {'A': 0.4171, 'B': 0.1821, 'C': 0.4007}
Uniform preference for all candidates {'A': 0.3079, 'B': 0.3069, 'C': 0.3852}


Let's initialize the s-PL model from the Dirichlet distribution, using that to build a preference interval rather than specifying the interval. Each bloc will need two Dirichlet alpha values; one to describe their own preference interval, and another to describe their preference for the opposing candidates.

In [17]:
bloc_voter_prop = {"X": 0.8, "Y": 0.2}

# the values of .9 indicate that these blocs are highly polarized;
# they prefer their own candidates much more than the opposing slate
cohesion_parameters = {"X": {"X": 0.9, "Y": 0.1}, "Y": {"Y": 0.9, "X": 0.1}}

alphas = {"X": {"X": 2, "Y": 1}, "Y": {"X": 1, "Y": 0.5}}

slate_to_candidates = {"X": ["X1", "X2"], "Y": ["Y1", "Y2"]}

# the from_params method allows us to sample from
# the Dirichlet distribution for our intervals
pl = bg.slate_PlackettLuce.from_params(
    slate_to_candidates=slate_to_candidates,
    bloc_voter_prop=bloc_voter_prop,
    cohesion_parameters=cohesion_parameters,
    alphas=alphas,
)

print("Preference interval for X bloc and X candidates")
print(pl.pref_intervals_by_bloc["X"]["X"])
print()
print("Preference interval for X bloc and Y candidates")
print(pl.pref_intervals_by_bloc["X"]["Y"])

print()
profile_dict, agg_profile = pl.generate_profile(number_of_ballots=100, by_bloc=True)
print(profile_df_head(profile_dict["X"],10))

Preference interval for X bloc and X candidates
{'X1': 0.2791, 'X2': 0.7209}

Preference interval for X bloc and Y candidates
{'Y1': 0.2134, 'Y2': 0.7866}

             Ranking_1 Ranking_2 Ranking_3 Ranking_4 Weight Voter Set
Ballot Index                                                         
6                 (X2)      (X1)      (Y2)      (Y1)     38        {}
0                 (X1)      (X2)      (Y2)      (Y1)     16        {}
7                 (X2)      (X1)      (Y1)      (Y2)     10        {}
2                 (Y2)      (X2)      (X1)      (Y1)      5        {}
8                 (X2)      (Y2)      (X1)      (Y1)      4        {}
4                 (Y2)      (X1)      (X2)      (Y1)      3        {}
1                 (X1)      (Y2)      (X2)      (Y1)      1        {}
3                 (Y2)      (Y1)      (X1)      (X2)      1        {}
5                 (X2)      (Y1)      (X1)      (Y2)      1        {}
9                 (Y1)      (X2)      (X1)      (Y2)      1        {}


Let's confirm that the intervals and ballots look reasonable. We have $\alpha_{XX} = 2$ and $\alpha_{XY} = 1$. This means that the $X$ voters tend to be relatively indifferent among their own candidates, but might adopt any candidate strength behavior for the $Y$ slate.


### **Try it yourself**
> Change the code above to check that the preference intervals and ballots for the $Y$ bloc look reasonable.



## Cambridge Sampler
We introduce one more method of generating ballots: the **Cambridge Sampler** (CS). CS generates ranked ballots using historical election data from Cambridge, MA (which has been continuously conducting ranked choice elections since 1941). It is the only ballot generator we will see today that is capable of producing incomplete ballots, including bullet votes.

By default, CS uses five elections (2009-2017, odd years); with the help of local organizers, we coded the candidates as White (W) or People of Color (POC, or C for short).  This is not necessarily the biggest factor predicting people's vote in Cambridge -- housing policy is the biggie -- but it's a good place to find realistic rankings, with candidates of two types.

You also have the option of providing CS with your own historical election data from which to generate ballots instead of using Cambridge data.

In [19]:
bloc_voter_prop = {"W": 0.8, "C": 0.2}

# the values of .9 indicate that these blocs are highly polarized;
# they prefer their own candidates much more than the opposing slate
cohesion_parameters = {"W": {"W": 0.9, "C": 0.1}, "C": {"C": 0.9, "W": 0.1}}

alphas = {"W": {"W": 2, "C": 1}, "C": {"W": 1, "C": 0.5}}

slate_to_candidates = {"W": ["W1", "W2", "W3"], "C": ["C1", "C2"]}

cs = bg.CambridgeSampler.from_params(
    slate_to_candidates=slate_to_candidates,
    bloc_voter_prop=bloc_voter_prop,
    cohesion_parameters=cohesion_parameters,
    alphas=alphas,
)


profile = cs.generate_profile(number_of_ballots=1000)
print(profile_df_head(profile,10).to_string())

             Ranking_1 Ranking_2 Ranking_3 Ranking_4 Ranking_5 Voter Set Weight
Ballot Index                                                                   
198               (C1)      (C2)      (W3)      (W1)      (W2)        {}     31
122               (W3)       (~)       (~)       (~)       (~)        {}     26
133               (W3)      (W1)      (W2)      (C2)      (C1)        {}     23
215               (C1)      (W3)      (C2)      (W1)      (W2)        {}     21
112               (W3)      (W2)      (W1)       (~)       (~)        {}     19
131               (W3)      (W1)      (W2)       (~)       (~)        {}     16
118               (W3)      (W2)      (C2)      (W1)      (C1)        {}     16
95                (W3)      (C2)      (C1)      (W2)      (W1)        {}     16
200               (C1)      (C2)       (~)       (~)       (~)        {}     15
85                (W1)      (W3)      (W2)      (C2)      (C1)        {}     14


Note: the ballot type (as in, Ws and Cs) is strictly drawn from the historical frequencies.  The candidate IDs (as in W1 and W2 among the W slate) are filled in by sampling without replacement from the preference interval that you either provided or made from Dirichlet alphas. That is the only role of the preference interval.

## Conclusion

There are many other models of ballot generation in VoteKit, both for ranked choice ballots and score based ballots (think cumulative or approval voting). See the [ballot generator](../../package_info/api.html#module-votekit.ballot_generator) section of the VoteKit documentation for more.

# Elections

Elections are the systems or algorithms by which a `PreferenceProfile`, or collection of ballots, is converted into an outcome. There are infinitely many different possible election methods, whether the output is a single winner, a set of winners, or a consensus ranking. VoteKit has a host of built-in election methods, as well as the functionality to let you create your own system of election. By the end of this section, you will have been introduced to the STV and Borda elections, learned about the `Election` object, and created your own election type.

## STV

To start, let's quickly introduce the Impartial Culture (IC) model to generate ballots for an STV election.  This model says that when a voter goes to cast a ranked-ballot, they do so by randomly choosing one out of all of the possible rankings (uniform over the set of permutations for you probabilty folk). That's it. There is only one knob to turn--the number of candidates--and no way to change the behavior of the voters. They all vote randomly.

This model might not be very realistic, but it is easy to use and prove theorems about, so it abounds in the academic literature. Impartial Culture is ill-suited to studying real-world jurisdictions.

In [ ]:
from votekit.elections import STV
from votekit.ballot_generator import ImpartialCulture

ic = ImpartialCulture(candidates = ["A", "B", "C", "D", "E"])
profile = ic.generate_profile(1000)

print(profile_df_head(profile, 10).to_string())

# m = 2 means 2 seats
election = STV(profile=profile, m=2)
print(election)

       Status  Round
A     Elected      3
C     Elected      3
D   Remaining      3
E  Eliminated      2
B  Eliminated      1


First, what is this showing?  Generally, the winners are listed in the order they were elected, from the top down.  Eliminated candidates are filled in in the order they were eliminated, bottom-up.  If any candidates are still remaining without having been designated elected or eliminated, they are in a middle category called `Remaining`.  Ties are broken by strength, meaning for instance that if 3 candidates are remaining at the end, they are listed in the order of their first-place in the final election state.  This means that this output can be thought of as an aggregate ranking vector produced by applying the election method to the voters' ranking vectors.  

So what exactly is happening in this STV election? STV stands for "single transferable vote." Voters cast ranked choice ballots. A threshold is set of how much support is required for election; if a candidate crosses the threshold, they are designated as a winner. The threshold in VoteKit defaults to something called the **Droop quota**. If there are $N$ voters and $m$ seats, then Droop quota is computed as $T=\lfloor N/(m+1)\rfloor +1$. Another option is the **Hare quota**, which is just $T=N/m$, which is a little bit larger. Generally, all that is needed of a threshold is that it can't be the case that $m+1$ candidates exceed it.

In the first round, the first-place votes for each candidate are tallied. If candidate $A$ crosses the threshold, they are elected. If there were surplus votes, then the ballots with $A$ in first place are transfered, with appropriately reduced weight, to the next choice of those voters. If another candidate receives enough transfered support to cross the threshold, they are elected. If no candidate does, the candidate with the fewest first-place votes is removed from all ballots, and their votes are transfered with full weight. This repeats until all seats are filled.

Let's work out a small example where it is easier to see how STV works. We will use a fractional transfer rule. If the threshold is $T$ and a candidate received $rT$ votes in a given round, where $r>1$, then the excess is $(r-1)T$ and so ballots are now "discounted" to have new weight $(r-1)/r$. For instance if the candidate received 150 votes but only needed 100, there would be 50 "excess" votes.  Instead of randomly picking 50 out of 150 ballots to transfer, we transfer them all with a reduced weight of 50/150, or 1/3.  Here is a [link](https://mggg.org/publications/political-geometry/20-WeighillDuchin.pdf) to a more substantial explainer about ranked choice.

In our example, suppose there are $N=23$ voters and $n=7$ candidates running for $m=3$ seats with the following profile.

In [3]:
from votekit.ballot import Ballot
from votekit.pref_profile import PreferenceProfile, profile_df_head

candidates = ["A", "B", "C", "D", "E", "F", "G"]

ballots = [
    Ballot(ranking=[{"A"}, {"B"}], weight=3),
    Ballot(ranking=[{"B"}, {"C"}, {"D"}], weight=8),
    Ballot(ranking=[{"C"}, {"A"}, {"B"}], weight=1),
    Ballot(ranking=[{"D"}, {"E"}], weight=3),
    Ballot(ranking=[{"E"}, {"D"}, {"F"}], weight=1),
    Ballot(ranking=[{"F"}, {"G"}], weight=4),
    Ballot(ranking=[{"G"}, {"E"}, {"F"}], weight=3),
]

profile = PreferenceProfile(ballots=ballots)

print(profile_df_head(profile,10))
print("Sum of ballot weights:", profile.total_ballot_wt)
print("Number of candidates:", len(profile.candidates))

election = STV(profile=profile, m=3)

print("Threshold:", election.threshold)
print("Number of rounds", len(election))
print(election)

             Ranking_1 Ranking_2 Ranking_3 Voter Set Weight
Ballot Index                                               
1                  (B)       (C)       (D)        {}      8
5                  (F)       (G)       (~)        {}      4
0                  (A)       (B)       (~)        {}      3
3                  (D)       (E)       (~)        {}      3
6                  (G)       (E)       (F)        {}      3
2                  (C)       (A)       (B)        {}      1
4                  (E)       (D)       (F)        {}      1
Sum of ballot weights: 23
Number of candidates: 7
Initial tiebreak was unsuccessful, performing random tiebreak
Threshold: 6
Number of rounds 6
       Status  Round
B     Elected      1
D     Elected      4
F     Elected      6
A   Remaining      6
G  Eliminated      5
C  Eliminated      3
E  Eliminated      2


What this code block did is create an `Election` object that lets us access all the information, round-by-round, about what would happen under the designated election method. The message about a tiebreak indicates that in some round, a random tiebreak was needed.

We can review it step-by-step instead of all at once. Just from a brief glance at the profile and threshold, we see that candidate B should be elected in the first round. Let's see this happen in two ways.

First, observe the first-place votes for each candidate. These are stored in the round 0 `ElectionState` object, which can be accessed as follows.

In [4]:
election.election_states[0].scores

{'A': Fraction(3, 1),
 'B': Fraction(8, 1),
 'C': Fraction(1, 1),
 'D': Fraction(3, 1),
 'E': Fraction(1, 1),
 'F': Fraction(4, 1),
 'G': Fraction(3, 1)}

We can see from this that only B is over the threshold.  The other way we can see who wins in the first round is by looking at the next `ElectionState`.



In [5]:
print("elected", election.election_states[1].elected)
print("\neliminated", election.election_states[1].eliminated)
print("\nremaining", election.election_states[1].remaining)

elected (frozenset({'B'}),)

eliminated (frozenset(),)

remaining (frozenset({'F'}), frozenset({'A', 'D', 'C', 'G'}), frozenset({'E'}))


$B$ passed the threshold by 2 votes with a total of 8, so the $B,C,D$ ballot is going to have $B$ removed and be given weight $2/8$ (excess/total) times its previous weight of 8. To check this, election objects have a method called `get_profile()` that returns the `PreferenceProfile` after a particular round.

In [7]:
pp = election.get_profile(1)
profile_df_head(pp, 10)

,Ranking_1,Ranking_2,Ranking_3,Voter Set,Weight
Ballot Index,,,,,
1,(F),(G),(~),{},4
3,(A),(~),(~),{},3
4,(D),(E),(~),{},3
6,(G),(E),(F),{},3
0,(C),(D),(~),{},2
2,(C),(A),(~),{},1
5,(E),(D),(F),{},1


Look, $B$ is now removed from all ballots, and the $B,C,D$ ballot became $C,D$ with weight 2. No one has enough votes to cross the 6 threshold, so the candidate with the least support will be eliminated---that is candidate $E$, with only one first-place vote.

We also introduce the `get_step()` method which accesses the profile and state of a given round.

In [9]:
print("fpv after round 1:", election.election_states[1].scores)
print("go to the next step\n")

profile, state = election.get_step(2)
print("elected", state.elected)
print("\neliminated", state.eliminated)
print("\nremaining", state.remaining)
print(profile_df_head(profile,10))

fpv after round 1: {'C': Fraction(3, 1), 'D': Fraction(3, 1), 'F': Fraction(4, 1), 'G': Fraction(3, 1), 'A': Fraction(3, 1), 'E': Fraction(1, 1)}
go to the next step

elected (frozenset(),)

eliminated (frozenset({'E'}),)

remaining (frozenset({'D', 'F'}), frozenset({'G', 'C', 'A'}))
             Ranking_1 Ranking_2 Ranking_3 Voter Set Weight
Ballot Index                                               
1                  (F)       (G)       (~)        {}      4
3                  (A)       (~)       (~)        {}      3
4                  (D)       (~)       (~)        {}      3
6                  (G)       (F)       (~)        {}      3
0                  (C)       (D)       (~)        {}      2
2                  (C)       (A)       (~)        {}      1
5                  (D)       (F)       (~)        {}      1


$E$ has been removed from all of the ballots. Again, no one crosses the threshold so the candidate with the fewest first-place votes will be eliminated.

In [11]:
print("fpv after round 2:", election.election_states[2].scores)
print("go to the next step\n")


print("elected", election.election_states[3].elected)
print("\neliminated", election.election_states[3].eliminated)
print("\nremaining", election.election_states[3].remaining)
print("\ntiebreak resolution", election.election_states[3].tiebreaks)
print()
print(profile_df_head(election.get_profile(3),10))

fpv after round 2: {'F': Fraction(4, 1), 'C': Fraction(3, 1), 'G': Fraction(3, 1), 'D': Fraction(4, 1), 'A': Fraction(3, 1)}
go to the next step

elected (frozenset(),)

eliminated (frozenset({'C'}),)

remaining (frozenset({'D'}), frozenset({'F', 'A'}), frozenset({'G'}))

tiebreak resolution {frozenset({'G', 'C', 'A'}): (frozenset({'A'}), frozenset({'G'}), frozenset({'C'}))}

Initial tiebreak was unsuccessful, performing random tiebreak
             Ranking_1 Ranking_2 Ranking_3 Voter Set Weight
Ballot Index                                               
1                  (F)       (G)       (~)        {}      4
3                  (A)       (~)       (~)        {}      3
4                  (D)       (~)       (~)        {}      3
6                  (G)       (F)       (~)        {}      3
0                  (D)       (~)       (~)        {}      2
2                  (A)       (~)       (~)        {}      1
5                  (D)       (F)       (~)        {}      1


Note that here, several candidates were tied for the fewest first-place votes at this stage. When this happens in STV, you use the first-place votes from the original profile to break ties. This means C will be eliminated. The `tiebreaks` parameter records the resolution of the tie; since we are looking for the person with the least first-place votes, the candidate in the final entry of the tuple is eliminated. The reason the message "Initial tiebreak was unsuccessful, performing random tiebreak" appeared is that A and G were tied by first-place votes, and thus a random tiebreak was needed to separate them. This didn't affect the outcome, since C had the fewest first-place votes.

### **Try it yourself**
> Keep printing the first-place votes and running a step of the election until all seats have been filled. At each step, think through why the election state transitioned as it did.

We now change the transfer type. Using the same profile as above, we'll now use `random_transfer`. In the default fractional transfer, we reweighted all of the ballots in proportion to the surplus. Here, we will randomly choose the appropriate number of ballots to transfer (the same number as the surplus).  Though it sounds strange, this is the method actually used in Cambridge, MA.  (Recall that Cambridge has used STV continuously since 1941 so back in the day they probably needed a low-tech physical way to do the transfers.)

In [12]:
from votekit.elections import random_transfer

candidates = ["A", "B", "C", "D", "E", "F", "G"]

ballots = [
    Ballot(ranking=[{"A"}, {"B"}], weight=3),
    Ballot(ranking=[{"B"}, {"C"}, {"D"}], weight=8),
    Ballot(ranking=[{"B"}, {"D"}, {"C"}], weight=8),
    Ballot(ranking=[{"C"}, {"A"}, {"B"}], weight=1),
    Ballot(ranking=[{"D"}, {"E"}], weight=1),
    Ballot(ranking=[{"E"}, {"D"}, {"F"}], weight=1),
    Ballot(ranking=[{"F"}, {"G"}], weight=4),
    Ballot(ranking=[{"G"}, {"E"}, {"F"}], weight=1),
]

profile = PreferenceProfile(ballots=ballots)

print(profile)
print("Sum of ballot weights:", profile.total_ballot_wt)
print("Number of candidates:", len(profile.candidates))

election = STV(profile=profile, transfer=random_transfer, m=2)

print(election)

Profile contains rankings: True
Maximum ranking length: 3
Profile contains scores: False
Candidates: ('A', 'B', 'C', 'D', 'E', 'F', 'G')
Candidates who received votes: ('A', 'B', 'C', 'D', 'E', 'F', 'G')
Total number of Ballot objects: 8
Total weight of Ballot objects: 27

Sum of ballot weights: 27
Number of candidates: 7
Initial tiebreak was unsuccessful, performing random tiebreak
       Status  Round
B     Elected      1
D     Elected      7
F  Eliminated      6
C  Eliminated      5
A  Eliminated      4
G  Eliminated      3
E  Eliminated      2


### **Try it yourself**
> Rerun the code above until you see that different candidates can win under random transfer.

## Election

Let's poke around the `Election` class a bit more. It contains a lot of useful information about what is happening in an election. We will also introduce the Borda election.

### Borda Election

In a Borda election, ranked ballots are converted to a score for a candidate, and then the candidates with the highest scores win. The traditional score vector is $(n,n-1,\dots,1)$: that is, if there are $n$ candidates, the first-place candidate on a ballot is given $n$ points, the second place $n-1$, all the way down to last, who is given $1$ point. You can change the score vector using the `score_vector` parameter.

Let's quickly introduce the Impartial Anonymous Culture (IAC) model to generate ballots for a Borda election. This is best described in terms of profiles rather than ballots. For a fixed number of ballots `n`, the IAC model generates a profile of size `n` uniformly at random from the set of all profiles. The IAC model only produces complete ballots. Again, the only parameterization is the number of candidates.

This model might not be very realistic, but it is easy to use and prove theorems about, so it abounds in the academic literature. Impartial Anonymous Culture is ill-suited to studying real-world jurisdictions.

In [13]:
from votekit.elections import Borda
import votekit.ballot_generator as bg

candidates = ["A", "B", "C", "D", "E", "F"]

# recall IAC generates an "all bets are off" profile
iac = bg.ImpartialAnonymousCulture(candidates=candidates)
profile = iac.generate_profile(number_of_ballots=1000)

election = Borda(profile, m=3)

In [17]:
print(profile_df_head(election.get_profile(0),10).to_string())
print()

print(election)

             Ranking_1 Ranking_2 Ranking_3 Ranking_4 Ranking_5 Ranking_6 Voter Set Weight
Ballot Index                                                                             
114                (B)       (C)       (E)       (D)       (A)       (F)        {}     16
49                 (B)       (A)       (C)       (E)       (D)       (F)        {}     11
97                 (D)       (F)       (B)       (A)       (C)       (E)        {}     10
64                 (D)       (B)       (A)       (F)       (C)       (E)        {}     10
22                 (A)       (B)       (C)       (E)       (D)       (F)        {}      9
28                 (C)       (A)       (D)       (F)       (E)       (B)        {}      9
2                  (C)       (A)       (E)       (F)       (D)       (B)        {}      8
94                 (C)       (E)       (B)       (D)       (F)       (A)        {}      8
13                 (B)       (C)       (F)       (E)       (D)       (A)        {}      8
30        

The Borda election is one-shot (like plurality), so running a step or the election is equivalent. Let's see what the election stores.



In [18]:
# the winners up to the given round, -1 means final round
print("Winners:", election.get_elected(-1))

# the eliminated candidates up to the given round
print("Eliminated:", election.get_eliminated(-1))

# the ranking of the candidates up to the given round
print("Ranking:", election.get_ranking(-1))

# the outcome of the given round
print("Outcome of round 1:\n", election.get_status_df(1))

Winners: (frozenset({'B'}), frozenset({'C'}), frozenset({'D'}))
Eliminated: ()
Ranking: (frozenset({'B'}), frozenset({'C'}), frozenset({'D'}), frozenset({'E'}), frozenset({'F'}), frozenset({'A'}))
Outcome of round 1:
       Status  Round
B    Elected      1
C    Elected      1
D    Elected      1
E  Remaining      1
F  Remaining      1
A  Remaining      1


### **Try it yourself**

> Using the following preference profile, try changing the score vector of a Borda election. Try replacing 3,2,1 with other Borda weights (decreasing and non-negative) showing that each candidate can be elected.

In [ ]:
ballots = [
    Ballot(ranking=[{"A"}, {"B"}, {"C"}], weight=3),
    Ballot(ranking=[{"A"}, {"C"}, {"B"}], weight=2),
    Ballot(ranking=[{"B"}, {"C"}, {"A"}], weight=2),
    Ballot(ranking=[{"C"}, {"B"}, {"A"}], weight=4),
]

profile = PreferenceProfile(ballots=ballots, candidates=["A", "B", "C"])

# borda election
score_vector = [3, 2, 1]
election = Borda(profile, m=1, score_vector=score_vector)
print(election)

      Status  Round
C    Elected      1
B  Remaining      1
A  Remaining      1


Since a Borda election is a one-shot election, most of the information stored in the `Election` is extraneous, but you can see its utility in an STV election where there are many rounds.

In [19]:
candidates = ["A", "B", "C", "D", "E", "F"]
iac = bg.ImpartialAnonymousCulture(candidates=candidates)
profile = iac.generate_profile(number_of_ballots=1000)


election = STV(profile=profile, m=1)

for i in range(1, 6):
    print(f"Round {i}\n")
    # the winners up to the current round
    print("Winners:", election.get_elected(i))

    # the eliminated candidates up to the current round
    print("Eliminated:", election.get_eliminated(i))

    # the remaining candidates, sorted by first-place votes
    print("Remaining:", election.get_remaining(i))

    # the same information as a df
    print(election.get_status_df(i))

    print()

Round 1

Winners: ()
Eliminated: (frozenset({'B'}),)
Remaining: (frozenset({'E', 'C'}), frozenset({'F'}), frozenset({'D'}), frozenset({'A'}))
       Status  Round
E   Remaining      1
C   Remaining      1
F   Remaining      1
D   Remaining      1
A   Remaining      1
B  Eliminated      1

Round 2

Winners: ()
Eliminated: (frozenset({'A'}), frozenset({'B'}))
Remaining: (frozenset({'E'}), frozenset({'C'}), frozenset({'F'}), frozenset({'D'}))
       Status  Round
E   Remaining      2
C   Remaining      2
F   Remaining      2
D   Remaining      2
A  Eliminated      2
B  Eliminated      1

Round 3

Winners: ()
Eliminated: (frozenset({'D'}), frozenset({'A'}), frozenset({'B'}))
Remaining: (frozenset({'E'}), frozenset({'C'}), frozenset({'F'}))
       Status  Round
E   Remaining      3
C   Remaining      3
F   Remaining      3
D  Eliminated      3
A  Eliminated      2
B  Eliminated      1

Round 4

Winners: ()
Eliminated: (frozenset({'F'}), frozenset({'D'}), frozenset({'A'}), frozenset({'B'}))


## Conclusion
There are many different possible election methods, both for choosing a single seat or multiple seats. `VoteKit` has a host of built-in election methods, as well as the functionality to let you create your own kind of election. You have been introduced to the STV and Borda elections and learned about the `Election` object. This should allow you to model any kind of elections you see in the real world, including rules that have not yet been implemented in `VoteKit`.